In [3]:
import zipfile
import os

def extract_files(zip_path):
    extract_dir = os.path.dirname(zip_path)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extracted all files from '{zip_path}' into '{extract_dir}'.")

extract_files("Training.zip")

Extracted all files from 'Training.zip' into ''.


In [1]:
import torch
import os
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

In [2]:
root_dir = "Training"

# image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # resizing for ResNet input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard ResNet normalization
])

dataset = datasets.ImageFolder(root=root_dir, transform=transform)

# splits training data into 3:1 ratio
train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# dataloaders for training
# batch_size = 32
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 2)

# for checking purposes
print(f"Total images: {len(dataset)}")
print(f"Training images: {len(train_dataset)}")
print(f"Validation images: {len(val_dataset)}")


/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Total images: 39375
Training images: 29531
Validation images: 9844


In [3]:
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

# sets device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# moves the model to device
model = model.to(device)

# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

# training/validation
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    avg_train_loss = train_loss / total_train
    train_accuracy = correct_train / total_train


    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validating"):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)


            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    avg_val_loss = val_loss / total_val
    val_accuracy = correct_val / total_val

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")


Validating: 100%|██████████| 616/616 [01:05<00:00,  9.40it/s]


Epoch [1/20], Train Loss: 0.0950, Train Accuracy: 0.9687, Val Loss: 0.5843, Val Accuracy: 0.6932


Validating: 100%|██████████| 616/616 [00:30<00:00, 20.32it/s]


Epoch [2/20], Train Loss: 0.0358, Train Accuracy: 0.9876, Val Loss: 0.0270, Val Accuracy: 0.9909


Validating: 100%|██████████| 616/616 [00:31<00:00, 19.47it/s]


...

Epoch [18/20], Train Loss: 0.0047, Train Accuracy: 0.9986, Val Loss: 0.0023, Val Accuracy: 0.9994


Validating: 100%|██████████| 616/616 [00:30<00:00, 19.98it/s]


Epoch [19/20], Train Loss: 0.0031, Train Accuracy: 0.9992, Val Loss: 0.0056, Val Accuracy: 0.9977


Validating: 100%|██████████| 616/616 [00:30<00:00, 19.87it/s]

Epoch [20/20], Train Loss: 0.0030, Train Accuracy: 0.9987, Val Loss: 0.0030, Val Accuracy: 0.9991


In [4]:
# for saving the model
model_save_path = "flame_classification.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to flame_classification.pth


In [5]:
# for loading the model
model_load_path = "flame_classification.pth"
model.load_state_dict(torch.load(model_load_path))
# model = model.to(device)

/local_scratch/slurm.1058835/ipykernel_1267829/2754892897.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_load_path))


<All keys matched successfully>

In [6]:
from torchvision import datasets
from torch.utils.data import DataLoader

# test dataset transformations
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


test_dir = "Test"
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# model testing
model.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct_test += (predicted == labels).sum().item()
        total_test += labels.size(0)

test_accuracy = correct_test / total_test
print(f"Test Accuracy: {test_accuracy:.4f}")

Testing: 100%|██████████| 270/270 [00:50<00:00,  5.37it/s]

Test Accuracy: 0.7042
